In [1]:
import pandas as pd 
df=pd.read_csv("full_data_fusionnnnne.csv")
df

,source,title,url,published_at,tags,content
0,France_24,الألعاب الأولمبية: الملاكمة خليف تبلغ ربع النه...,https://www.france24.com/ar/%D8%B1%D9%8A%D8%A7...,2024-08-01,رياضة,تغلبت الملاكمة الجزائرية إيمان خليف على خصمتها...
1,France_24,"محطة أفريقيا في الألعاب الأولمبية بباريس: ""خيم...",https://www.france24.com/ar/%D8%B1%D9%8A%D8%A7...,2024-08-01,رياضة,موفد فرانس24 إلى سان دوني – تتواصل فعاليات معر...
2,France_24,تنديدا باغتيال هنية... مظاهرات مناهضة لإسرائيل...,https://www.france24.com/ar/%D8%A7%D9%84%D8%B4...,2024-08-01,الشرق الأوسط,تظاهر آلاف الأشخاص في تركيا والمغرب وتونس تندي...
3,France_24,"إسرائيل ""تغتال"" صحفيا ومصورا من قناة الجزيرة ف...",https://www.france24.com/ar/%D9%81%D9%8A%D8%AF...,2024-08-01,الأخبار المغاربية,"دانت قناة الجزيرة القطرية الأربعاء ""اغتيال"" اث..."
4,France_24,خليل الحية عن اغتيال هنية.. هذه جريمة لا بد لم...,https://www.france24.com/ar/%D9%81%D9%8A%D8%AF...,2024-08-01,الأخبار المغاربية,خليل الحية القيادي في حماس: الحركة وإيران لا ت...
...,...,...,...,...,...,...
41549,Al Hurra,قوة بحرية: قراصنة يستولون على سفينة صيد صينية ...,https://www.alhurra.com/arabic-and-internation...,2024-12-05,NO_TAGS,أعلنت وزارة الصحة الفلسطينية في قطاع غزة، الأر...
41550,Al Hurra,"بعد كاليفورنيا.. ولاية أميركية أخرى تستعد ""لحظ...",https://www.alhurra.com/usa/2024/11/30/%D9%83%...,2024-11-30,NO_TAGS,جدد الرئيس الأميركي دونالد ترامب، الأربعاء، ان...
41551,Al Hurra,ترامب: لقد كتبنا التاريخ,https://www.alhurra.com/us-election-2024/2024/...,2024-11-06,NO_TAGS,أعلن المرشح الجمهوري لانتخابات الرئاسة الأميرك...
41552,Al Hurra,"حقيقة فيديو ""بيع أصنام"" في السعودية",https://www.alhurra.com/hl-hqa/2024/10/27/%D8%...,2024-10-27,NO_TAGS,تداولت صفحات وحسابات على مواقع التواصل الاجتما...


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import arabic_reshaper
from bidi.algorithm import get_display
from scipy.sparse import csr_matrix

# --- 1. Filtrer les NO_TAGS ---------------------------------
df_filtered = df[df['tags'] != 'NO_TAGS'].reset_index(drop=True)

# --- 2. Fonction pour reshaper les titres arabes ------------
def reshape_arabic(text: str) -> str:
    """Reshape et reorder pour un affichage correct de l'arabe."""
    return get_display(arabic_reshaper.reshape(text))

# --- 3. Construire la matrice TF-IDF sur 'content' -----------
vectorizer = TfidfVectorizer(max_features=500, stop_words=None)
tfidf_matrix = vectorizer.fit_transform(df_filtered['content'])

# --- 4. Calculer la similarité cosinus par lots -------------
N_VOISINS = 3  # Nombre de voisins

# Liste pour stocker les résultats
results = []

# Traiter par morceaux pour éviter des calculs trop lourds
batch_size = 100  # Traiter 100 articles à la fois

for start_idx in range(0, len(df_filtered), batch_size):
    end_idx = min(start_idx + batch_size, len(df_filtered))
    
    # Calculer la similarité cosinus pour le sous-ensemble
    sim_matrix_batch = cosine_similarity(tfidf_matrix[start_idx:end_idx], tfidf_matrix)
    
    for i in range(start_idx, end_idx):
        # Classement des documents selon leur similarité avec i
        neighbors = sorted(
            enumerate(sim_matrix_batch[i - start_idx]),
            key=lambda x: x[1],
            reverse=True
        )[1: N_VOISINS + 1]  # on enlève soi-même (score=1.0)
        
        for j, score in neighbors:
            results.append({
                'article_idx': i,
                'article_title': reshape_arabic(df_filtered.at[i, 'title']),
                'neighbor_idx': j,
                'neighbor_title': reshape_arabic(df_filtered.at[j, 'title']),
                'similarity_score': score
            })

# --- 5. Mettre dans un DataFrame ----------------------------
neighbors_df = pd.DataFrame(results)

# Réorganiser les colonnes pour plus de clarté
neighbors_df = neighbors_df[['article_idx', 'article_title', 'neighbor_idx', 'neighbor_title', 'similarity_score']]

# --- Affichage ---------------------------------------------
# Afficher les 20 premières lignes
print(neighbors_df.head(20))

# Optionnel : sauvegarder dans un fichier CSV
# neighbors_df.to_csv('tfidf_neighbors.csv', index=False)


    article_idx                                      article_title  \
0             0  ﺎﻫﺪﺿ "ﺔﻠﻃﺎﺒﻟﺍ" ﺔﻳﺎﻋﺪﻟﺍ ﺮﻜﻨﺘﺴﺗ ﺮﺋﺍﺰﺠﻟﺍﻭ ﻲﺋﺎﻬﻨﻟﺍ...   
1             0  ﺎﻫﺪﺿ "ﺔﻠﻃﺎﺒﻟﺍ" ﺔﻳﺎﻋﺪﻟﺍ ﺮﻜﻨﺘﺴﺗ ﺮﺋﺍﺰﺠﻟﺍﻭ ﻲﺋﺎﻬﻨﻟﺍ...   
2             0  ﺎﻫﺪﺿ "ﺔﻠﻃﺎﺒﻟﺍ" ﺔﻳﺎﻋﺪﻟﺍ ﺮﻜﻨﺘﺴﺗ ﺮﺋﺍﺰﺠﻟﺍﻭ ﻲﺋﺎﻬﻨﻟﺍ...   
3             1  "ﺀﺍﺮﻤﺴﻟﺍ ﺓﺭﺎﻘﻟﺍ ﺀﺍﺮﺛ ﻦﻋ ﺮﺒﻌﺗ ﺪﻠﺑ ﻞﻛ ﺔﻤﻴﺧ" :ﺲﻳﺭ...   
4             1  "ﺀﺍﺮﻤﺴﻟﺍ ﺓﺭﺎﻘﻟﺍ ﺀﺍﺮﺛ ﻦﻋ ﺮﺒﻌﺗ ﺪﻠﺑ ﻞﻛ ﺔﻤﻴﺧ" :ﺲﻳﺭ...   
5             1  "ﺀﺍﺮﻤﺴﻟﺍ ﺓﺭﺎﻘﻟﺍ ﺀﺍﺮﺛ ﻦﻋ ﺮﺒﻌﺗ ﺪﻠﺑ ﻞﻛ ﺔﻤﻴﺧ" :ﺲﻳﺭ...   
6             2  ﻁﺎﺑﺮﻟﺍﻭ ﺲﻧﻮﺗﻭ ﻝﻮﺒﻨﻄﺳﺇ ﻲﻓ ﻞﻴﺋﺍﺮﺳﻹ ﺔﻀﻫﺎﻨﻣ ﺕﺍﺮﻫﺎﻈ...   
7             2  ﻁﺎﺑﺮﻟﺍﻭ ﺲﻧﻮﺗﻭ ﻝﻮﺒﻨﻄﺳﺇ ﻲﻓ ﻞﻴﺋﺍﺮﺳﻹ ﺔﻀﻫﺎﻨﻣ ﺕﺍﺮﻫﺎﻈ...   
8             2  ﻁﺎﺑﺮﻟﺍﻭ ﺲﻧﻮﺗﻭ ﻝﻮﺒﻨﻄﺳﺇ ﻲﻓ ﻞﻴﺋﺍﺮﺳﻹ ﺔﻀﻫﺎﻨﻣ ﺕﺍﺮﻫﺎﻈ...   
9             3  ﺓﺰﻏ ﻉﺎﻄﻗ ﻲﻓ ﺓﺮﻳﺰﺠﻟﺍ ﺓﺎﻨﻗ ﻦﻣ ﺍﺭﻮﺼﻣﻭ ﺎﻴﻔﺤﺻ "ﻝﺎﺘﻐ...   
10            3  ﺓﺰﻏ ﻉﺎﻄﻗ ﻲﻓ ﺓﺮﻳﺰﺠﻟﺍ ﺓﺎﻨﻗ ﻦﻣ ﺍﺭﻮﺼﻣﻭ ﺎﻴﻔﺤﺻ "ﻝﺎﺘﻐ...   
11            3  ﺓﺰﻏ ﻉﺎﻄﻗ ﻲﻓ ﺓﺮﻳﺰﺠﻟﺍ ﺓﺎﻨﻗ ﻦﻣ ﺍﺭﻮﺼﻣﻭ ﺎﻴﻔﺤﺻ "ﻝﺎﺘﻐ...   
12            4  ﻦﻤﺜﻟﺍ ﻊﻓﺪﻳ ﻥﺃ ﺎﻬﺒﻜﺗﺮﻤﻟ ﺪﺑ ﻻ ﺔﻤﻳﺮﺟ ﻩﺬﻫ ..ﺔﻴﻨﻫ ﻝ...   
13            4  ﻦﻤﺜ

In [3]:
# Appliquer reshape_arabic à tous les titres dans neighbors_df pour les afficher correctement
neighbors_df['article_title'] = neighbors_df['article_title'].apply(reshape_arabic)
neighbors_df['neighbor_title'] = neighbors_df['neighbor_title'].apply(reshape_arabic)

# Afficher les 20 premières lignes pour vérifier
print(neighbors_df.head(20))



    article_idx                                      article_title  \
0             0  ﺍﻷﻟﻌﺎﺏ ﺍﻷﻭﻟﻤﺒﻴﺔ: ﺍﻟﻤﻼﻛﻤﺔ ﺧﻠﻴﻒ ﺗﺒﻠﻎ ﺭﺑﻊ ﺍﻟﻨﻬﺎﺋﻲ...   
1             0  ﺍﻷﻟﻌﺎﺏ ﺍﻷﻭﻟﻤﺒﻴﺔ: ﺍﻟﻤﻼﻛﻤﺔ ﺧﻠﻴﻒ ﺗﺒﻠﻎ ﺭﺑﻊ ﺍﻟﻨﻬﺎﺋﻲ...   
2             0  ﺍﻷﻟﻌﺎﺏ ﺍﻷﻭﻟﻤﺒﻴﺔ: ﺍﻟﻤﻼﻛﻤﺔ ﺧﻠﻴﻒ ﺗﺒﻠﻎ ﺭﺑﻊ ﺍﻟﻨﻬﺎﺋﻲ...   
3             1  ﻣﺤﻄﺔ ﺃﻓﺮﻳﻘﻴﺎ ﻓﻲ ﺍﻷﻟﻌﺎﺏ ﺍﻷﻭﻟﻤﺒﻴﺔ ﺑﺒﺎﺭﻳﺲ: "ﺧﻴﻤﺔ ...   
4             1  ﻣﺤﻄﺔ ﺃﻓﺮﻳﻘﻴﺎ ﻓﻲ ﺍﻷﻟﻌﺎﺏ ﺍﻷﻭﻟﻤﺒﻴﺔ ﺑﺒﺎﺭﻳﺲ: "ﺧﻴﻤﺔ ...   
5             1  ﻣﺤﻄﺔ ﺃﻓﺮﻳﻘﻴﺎ ﻓﻲ ﺍﻷﻟﻌﺎﺏ ﺍﻷﻭﻟﻤﺒﻴﺔ ﺑﺒﺎﺭﻳﺲ: "ﺧﻴﻤﺔ ...   
6             2  ﺗﻨﺪﻳﺪﺍ ﺑﺎﻏﺘﻴﺎﻝ ﻫﻨﻴﺔ... ﻣﻈﺎﻫﺮﺍﺕ ﻣﻨﺎﻫﻀﺔ ﻹﺳﺮﺍﺋﻴﻞ ...   
7             2  ﺗﻨﺪﻳﺪﺍ ﺑﺎﻏﺘﻴﺎﻝ ﻫﻨﻴﺔ... ﻣﻈﺎﻫﺮﺍﺕ ﻣﻨﺎﻫﻀﺔ ﻹﺳﺮﺍﺋﻴﻞ ...   
8             2  ﺗﻨﺪﻳﺪﺍ ﺑﺎﻏﺘﻴﺎﻝ ﻫﻨﻴﺔ... ﻣﻈﺎﻫﺮﺍﺕ ﻣﻨﺎﻫﻀﺔ ﻹﺳﺮﺍﺋﻴﻞ ...   
9             3  ﺇﺳﺮﺍﺋﻴﻞ "ﺗﻐﺘﺎﻝ" ﺻﺤﻔﻴﺎ ﻭﻣﺼﻮﺭﺍ ﻣﻦ ﻗﻨﺎﺓ ﺍﻟﺠﺰﻳﺮﺓ ﻓ...   
10            3  ﺇﺳﺮﺍﺋﻴﻞ "ﺗﻐﺘﺎﻝ" ﺻﺤﻔﻴﺎ ﻭﻣﺼﻮﺭﺍ ﻣﻦ ﻗﻨﺎﺓ ﺍﻟﺠﺰﻳﺮﺓ ﻓ...   
11            3  ﺇﺳﺮﺍﺋﻴﻞ "ﺗﻐﺘﺎﻝ" ﺻﺤﻔﻴﺎ ﻭﻣﺼﻮﺭﺍ ﻣﻦ ﻗﻨﺎﺓ ﺍﻟﺠﺰﻳﺮﺓ ﻓ...   
12            4  ﺧﻠﻴﻞ ﺍﻟﺤﻴﺔ ﻋﻦ ﺍﻏﺘﻴﺎﻝ ﻫﻨﻴﺔ.. ﻫﺬﻩ ﺟﺮﻳﻤﺔ ﻻ ﺑﺪ ﻟﻤﺮ...   
13            4  ﺧﻠﻴ